In [1]:
import os
import textwrap
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


In [7]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
from datasets import load_dataset
ds = load_dataset(
    "hwaseem04/Aya-testing",
    data_files={"xm3600_captioning": "data/xm3600_captioning-00000-of-00001.parquet"}
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


xm3600_captioning-00000-of-00001.parquet:   0%|          | 0.00/197M [00:00<?, ?B/s]

Generating xm3600_captioning split: 0 examples [00:00, ? examples/s]

In [5]:
ds['xm3600_captioning'][0]

{'sample_id': 1,
 'image_id': '000411001ff7dd4f',
 'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=640x480>,
 'prompt_en': 'Caption the image, short answer.',
 'captions_en': ['a rooster and hens surrounded by green leaves .',
  'a rooster with two hens on a rocky slope with some bushes .'],
 'prompt_ar': 'علّق على الصورة، بإجابة قصيرة.',
 'captions_ar': ['ديك و فرخة علي الأرض', 'ديك وفرخة بحديقة'],
 'prompt_bn': 'ছবির ক্যাপশন দিন, সংক্ষিপ্ত উত্তর।',
 'captions_bn': ['জঙ্গলের মদহে দুইটি বাদামী ও কালো রঙের মুরগি আছে'],
 'prompt_cs': 'Popište obrázek krátkou odpovědí.',
 'captions_cs': ['kohout a slepice v trávě',
  'hnědá slepice a kohout jdoucí po trávě v lese'],
 'prompt_da': 'Giv billedet en billedtekst, kort svar.',
 'captions_da': ['en brun høne og en flerfarvet hane i skovbunden',
  'hane og høne i have'],
 'prompt_de': 'Bildunterschrift für das Bild, kurze Antwort.',
 'captions_de': ['eine henne und ein hanh im steinigem garten im gras .',
  'leicht verschwommene 

In [8]:
from tqdm import tqdm
from PIL import Image
import torch
from transformers import AutoProcessor, Gemma3ForConditionalGeneration

# Load Gemma model and processor
model_id = "google/gemma-3-4b-it"
model = Gemma3ForConditionalGeneration.from_pretrained(model_id, device_map="auto").eval()
processor = AutoProcessor.from_pretrained(model_id)

# Languages to iterate over
languages = ["en", "bn", "de", "ko", "ru", "zh"]

# Directory to save temp images (needed for this model)
os.makedirs("temp_images_caption", exist_ok=True)

dataset = ds['xm3600_captioning']

for sample in tqdm(dataset, desc="Iterating samples"):
    try:
        image = sample["image"]
        sample_id = sample["sample_id"]

        # Save image locally if it doesn't already exist
        image_path = f"temp_images_caption/{sample_id}.jpg"
        if not os.path.exists(image_path):
            image.save(image_path)

        print(f"\n========== Sample ID: {sample_id} ==========")

        for lang in languages:
            prompt_col = f"prompt_{lang}"
            caption_col = f"captions_{lang}"

            # Skip if required fields are missing
            if prompt_col not in sample or caption_col not in sample:
                print(f"[{lang}] Missing data.")
                continue

            prompt = sample[prompt_col]
            gt_caption = sample[caption_col]

            # Build the prompt using Gemma's chat template style
            messages = [
                {
                    "role": "system",
                    "content": [{"type": "text", "text": "You are a helpful assistant."}]
                },
                {
                    "role": "user",
                    "content": [
                        {"type": "image", "image": image_path},
                        {"type": "text", "text": prompt}
                    ]
                }
            ]

            # Prepare input using Gemma's processor
            inputs = processor.apply_chat_template(
                messages,
                add_generation_prompt=True,
                tokenize=True,
                return_dict=True,
                return_tensors="pt"
            ).to(model.device, dtype=torch.bfloat16)

            input_len = inputs["input_ids"].shape[-1]

            # Generate output from model
            with torch.inference_mode():
                generation = model.generate(
                    **inputs,
                    max_new_tokens=100,
                    do_sample=False
                )
                generation = generation[0][input_len:]

            # Decode the generated tokens
            pred_caption = processor.decode(generation, skip_special_tokens=True)

            # Display results for this language
            print(f"\n[{lang.upper()}]")
            print(f"Prompt: {prompt}")
            print(f"GT: {gt_caption}")
            print(f"Pred: {textwrap.fill(pred_caption, width=80)}")

        print("=" * 100)

    except Exception as e:
        print(f"Error processing sample {sample['sample_id']}: {e}")

    break

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/90.6k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.64G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

Iterating samples:   0%|          | 0/3600 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:653: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `64` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(



========== Sample ID: 1 ==========

[EN]
Prompt: Caption the image, short answer.
GT: ['a rooster and hens surrounded by green leaves .', 'a rooster with two hens on a rocky slope with some bushes .']
Pred: Here are a few short captions for the image:  *   A flock of chickens in a
natural setting. *   Rooster and hens exploring the outdoors. *   Farm life at
its finest.

[BN]
Prompt: ছবির ক্যাপশন দিন, সংক্ষিপ্ত উত্তর।
GT: ['জঙ্গলের মদহে দুইটি বাদামী ও কালো রঙের মুরগি আছে']
Pred: এখানে ছবির ক্যাপশন দেওয়া হলো:  "একটি মুরগি, একটি দেশাতী এবং আরও একটি মুরগি।"

[DE]
Prompt: Bildunterschrift für das Bild, kurze Antwort.
GT: ['eine henne und ein hanh im steinigem garten im gras .', 'leicht verschwommene aufnahme von huhn und hahn im freien tagsüber .']
Pred: Hier sind ein paar Bildunterschriften für das Bild:  *   Hühner im Freien *
Ein Hühnerhof *   Hühner auf der Jagd

[KO]
Prompt: 이미지에 캡션을 달아주세요. 짧은 답변으로.
GT: ['낙엽 과 잡초 가 많은 산길 의 돌 위에 서 있는 닭 두 마리', '시골 마을 에 풀어 놓고 키우는 닭 암수']
Pred: 닭들이 숲속에서 돌

Iterating samples:   0%|          | 0/3600 [00:14<?, ?it/s]


[ZH]
Prompt: 为图片添加标题，简短回答。
GT: ['在 山里 中 站着 两只鸡 ， 一只 黄色 另一 只 黑 黄色 ， 它们 俩 站着 看 向 同 一个 方向', '在 野外 绿植 地上 的 公鸡 和 母鸡 近景']
Pred: 当然，以下是图片的一些标题：  *   鸡群 *   乡村鸡 *   鸡和公鸡
